<a href="https://colab.research.google.com/github/GuJiMao/GNN_Learning/blob/main/%E5%9B%BE%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E5%85%A5%E9%97%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装

## torch-scatter

In [5]:
pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 2.6MB 6.7MB/s 


## torch-sparse

In [6]:
pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 1.5MB 7.6MB/s 


## torch-cluster

In [7]:
pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 1.0MB 6.8MB/s 


## torch-spline-conv

In [8]:
pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 389kB 6.7MB/s 


## torch_geometric

In [9]:
pip install torch_geometric

     |████████████████████████████████| 194kB 8.2MB/s 
     |████████████████████████████████| 235kB 10.9MB/s 
     |████████████████████████████████| 2.2MB 8.7MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.3-cp37-none-any.whl size=322719 sha256=563fdebfd1091ae42faf5b075775f1ce6e95719c259c1cd3698ad7c19b4147ea
  Stored in directory: /root/.cache/pip/wheels/6d/47/1e/0af8ce3e21783c3e584c22502011a3367c091694eebc50a971
Successfully built torch-geometric


# 说明
这部分的教程来自于PyTorch geometric官网，[点击链接](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html)进入

# 通过例子入门
接下来通过一些简单的例子来简单介绍一下PyTorch Geometric中的核心概念，PyTorch Geometric的主要特征如下：

## 图的数据处理
这个图和我们的数据结构中使用到的图一样，在各个节点之间通过边连接起来，通过`torch_geometric.data.data`这个实例就能描述出PyTorch Geometric中的一个简单的图，默认的带有以下属性
- `data.x`：节点特征矩阵，大小为`[num_nodes, num_node_features]`
- `data.edge_index`：以大小`[2, num_edges]`连接的图，类型为`torch.long`
- `data.edge_attr`：边缘特征矩阵，大小为`[num_edges, num_edge_features]`
- `data.y`：要训练对抗的目标，形状可能任意。有的大小可能为`[num_nodes, *]`，有的目标可能为`[, *]`
- `data.ps`：节点的位置矩阵，大小为`[num_nodes, num_dimensions]`
这些属性并不是必须的，事实上`data`这个对象并不局限于这些属性。例如，我们可以通过`data.face`来保存在张量中保存三维网格三角形的连通性，大小为`[3,num_faces]`，类型为`torch.long`。

下面展示一个图，它带有3个节点和4条边，但是无权重和无方向。每个节点事实上只含有一个特征

In [15]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long) # 即 0<->1, 1<->2，两个双向所以为无向
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

我们最后得到的data就如下图所示

![图片](https://pytorch-geometric.readthedocs.io/en/latest/_images/graph.svg)

***注意：***其中的`edge_index`，这个tensor向量定义了源节点和目标域节点的所有边，不是索引元组组成的列表。如果想要写成自己的索引，应该将它专职，然后再它进入数据构造器之前调用`contiguous`函数。

In [17]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],[1, 0],[1, 2],[2, 1]], dtype=torch.long) # 同样为0<->1, 1<->2
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())


通过上面的例子我们可以看到，只有2个点，4条边，但是矩阵却用了4个,$[0,1], [1,0], [1,2], [2,1]$
下面我们可以打印一下关于这个`data`的属性

In [19]:
print(data.keys)      # 通过结果可以看到，data是一个字典，data[节点: x, 边: edge_index]

['x', 'edge_index']


In [20]:
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [21]:
for key, item in data:
    print("{} found in data".format(key))

edge_index found in data
x found in data


In [22]:
data.num_nodes    # 图的节点数

3

In [23]:
data.num_edges    # 图的边数

4

In [24]:
data.num_node_features    # 节点的特征个数

1

In [26]:
data.contains_isolated_nodes() # 包含孤立点？

False

In [25]:
data.contains_self_loops()  # 图中包含循环？

False

In [27]:
data.is_directed()    # 图是有向的？

False

In [ ]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

这些方法都可以在`torch.geometric.data.Data`中找到

## 常见基准数据集

PyTorch Geometric包含许多常见的数据集。例如：所有行星的数据集（Cora, Citeseer, Pubmed），所有用于图分类的数据集，这里是它们的[链接](https://ls11-www.cs.tu-dortmund.de/staff/morris/graphkerneldatasets)和[纯净版的链接](https://github.com/nd7141/graph_datasets)，QM7和QM9数据集，少量的点云数据集，例如：FAUST, ModelNet10/40和ShapeNet

### ENZYMES数据集

初始化数据集很简单。数据集的初始化将自动下载其原始文件并将其处理为前面描述的`Data`格式。例如，加载ENZYMES数据集（包含600个图，含有6各类）

In [29]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
print('图的大小{0},类的个数{1},节点的特征个数{2}'.format(len(dataset), dataset.num_classes, dataset.num_node_features))

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!
图的大小600,类的个数6,节点的特征个数3


我们现在得到了这600个图

In [32]:
data = dataset[0]
print(data)
data.is_undirected()
print('图的节点数{0}，图的边数{1}，图的特征个数{2}'.format(data.num_nodes,data.num_edges,data.num_node_features))    # 图的节点数
print('是否包含孤立点{0}，是否包含循环{1}'.format(data.contains_isolated_nodes(), data.contains_self_loops()))

Data(edge_index=[2, 168], x=[37, 3], y=[1])
图的节点数37，图的边数168，图的特征个数3
是否包含孤立点False，是否包含循环False


我们可以通过切片索引，或者tensor向量来将我们的数据分成训练集和测试集

In [33]:
train_dataset = dataset[:540]
test_dataset = dataset[540:]

可以在将数据分割之前先打乱一下

In [34]:
# 打乱方法一：
dataset = dataset.shuffle()
# 打乱方法二：
perm = torch.randperm(len(dataset))
dataset = dataset[perm]

### Cora数据集

尝试另一个数据集：Cora，这个是一个用于半监督图节点分类的标准数据集

In [37]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
print('图的大小{0},类的个数{1},节点的特征个数{2}'.format(len(dataset),dataset.num_classes,dataset.num_node_features))

图的大小1,类的个数7,节点的特征个数1433


这个数据集是一个无向的，且只有一个图的数据集

In [38]:
data = dataset[0]
data.is_undirected()
data.train_mask.sum().item()
data.val_mask.sum().item()
data.test_mask.sum().item()

1000

这次，这个`Data`对象的每个节点都有了一个标签，并且还有额外的属性：`train_mask`,`val_mask`,`test_mask`
- `train_mask`：表示要训练的节点（140个节点）
- `val_mask`：用于验证的节点，可以用于提早停止（500个节点）
- `test_mask`：用于验证的节点（1000个节点）

## 小批次

神经网络经常小批次的训练数据，PyTorch Geometric通过创建稀疏块对角邻接矩阵（参数中的`edge_index`）在小批上实现并行化，然后在节点维度上拼接特征和目标矩阵，这种组合允许在一批示例中使用不同数量的节点和边。
$$\begin{split}\mathbf{A} = \begin{bmatrix} \mathbf{A}_1 & & \\ & \ddots & \\ & & \mathbf{A}_n \end{bmatrix}, \qquad \mathbf{X} = \begin{bmatrix} \mathbf{X}_1 \\ \vdots \\ \mathbf{X}_n \end{bmatrix}, \qquad \mathbf{Y} = \begin{bmatrix} \mathbf{Y}_1 \\ \vdots \\ \mathbf{Y}_n \end{bmatrix}\end{split}$$
PyTorch Geometric有它自己的`torch_geometric.data.DataLoader`，其中就有拼接操作，接下来看一下个例子。

In [42]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
for batch in loader:
  print(batch)
  print(batch.num_graphs)   # 图的个数

Batch(batch=[958], edge_index=[2, 3776], x=[958, 21], y=[32])
32
Batch(batch=[1187], edge_index=[2, 4346], x=[1187, 21], y=[32])
32
Batch(batch=[1193], edge_index=[2, 4424], x=[1193, 21], y=[32])
32
Batch(batch=[1095], edge_index=[2, 4174], x=[1095, 21], y=[32])
32
Batch(batch=[1056], edge_index=[2, 3680], x=[1056, 21], y=[32])
32
Batch(batch=[1039], edge_index=[2, 3860], x=[1039, 21], y=[32])
32
Batch(batch=[1071], edge_index=[2, 4138], x=[1071, 21], y=[32])
32
Batch(batch=[954], edge_index=[2, 3634], x=[954, 21], y=[32])
32
Batch(batch=[1027], edge_index=[2, 3916], x=[1027, 21], y=[32])
32
Batch(batch=[973], edge_index=[2, 3764], x=[973, 21], y=[32])
32
Batch(batch=[933], edge_index=[2, 3570], x=[933, 21], y=[32])
32
Batch(batch=[1130], edge_index=[2, 4142], x=[1130, 21], y=[32])
32
Batch(batch=[1120], edge_index=[2, 4496], x=[1120, 21], y=[32])
32
Batch(batch=[1057], edge_index=[2, 4174], x=[1057, 21], y=[32])
32
Batch(batch=[979], edge_index=[2, 3884], x=[979, 21], y=[32])
32
Batch

`torch_geometric.data.Batch`继承自`torch_geometric.data.Data`，包含了一个额外的属性叫做`batch`。

`batch`是一个列向量，将每个节点映射到`batch`中各自的图上:
$$\mathrm{batch} = {\begin{bmatrix} 0 & \cdots & 0 & 1 & \cdots & n - 2 & n -1 & \cdots & n - 1 \end{bmatrix}}^{\top}$$

可以在每个图中的节点维度上来平均节点特征（不太懂这句话）

In [ ]:
from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
for data in loader:
    print(data)
    print(data.num_graphs)
    x = scatter_mean(data.x, data.batch, dim=0)
    print(x.size())

你可以[在这里](https://pytorch-geometric.readthedocs.io/en/latest/notes/batching.html)了解更多关于PyTorch Geometric内部批处理过程的信息，例如，如何修改它的行为。关于分散操作的文档，有兴趣的读者可以参考`torch-scatter`[文档](https://pytorch-scatter.readthedocs.io/)。

## 数据转换

变换是`torchvision`中对图像进行变换和增强的常用方法。PyTorch Geometric附带了自己的转换，它期望一个`Data`对象作为输入，并返回一个新的转换后的`Data`对象。变换可以使用`torch_geometry.Transforms.Compose`链接在一起。在将已处理的数据集保存到磁盘(`pre_transform`)或访问数据集(`transform`)中的图之前应用合成和。

让我们看一个例子，在ShapeNet数据集(包含17000个3D形状点云和来自16个形状类别的每个点标签)上应用变换。

In [43]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

data = dataset[0]
print(data)

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!
Data(category=[1], pos=[2518, 3], x=[2518, 3], y=[2518])


我们可以将点云数据集转换为一个图数据集，通过变换从点云生成最近的邻居图:

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))

data = dataset[0]
print(data)

***注意***
> 在将数据保存到磁盘之前，我们使用pre_transform对数据进行转换(从而缩短加载时间)。请注意，下一次初始化数据集时，即使您没有传递任何转换，它也将已经包含图形边。

此外，我们可以使用`transform`参数来随机增加一个数据对象，例如，将每个节点的位置平移一个小的数字:

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomTranslate(0.01))

data = dataset[0]
print(data)

可以在`torch_geometric.transforms`中找到完整的变换

## 图网络的学习过程

在学完数据的加载，变换以后，接下来就是图神经网络的实现过程了。

使用一个简单的GCN层，在Cora引用数据集上复制实。对于更多的关于GCN的解释，[在这个博客中观看](http://tkipf.github.io/graph-convolutional-networks/)。

载入数据

In [44]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

我们接下来并不需要`transform`或者`dataloader`，现在先做一个两层的GCN

In [45]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16, dataset.num_classes)
  def forward(self, data):
    x, edge_index = data.x, data.edge_index
    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)

构造函数定义了两个`GCNConv`层，它们在网络的前向传递过程中被调用。请注意，非线性没有集成到`conv`调用中，因此需要在之后应用(这在PyTorch Geometric中的所有操作符中都是一致的)。在这里，我们选择使用`ReLU`作为我们的中间非线性，并最终在类的数量上输出一个softmax分布。训练这个模型200个轮回:

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

最后在验证节点上验证模型

In [47]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7920


至此，第一个图神经网络已经学习完成了。最简单的学习图神经网络的方式就是学习文件夹`examples/`中的例子，然后浏览`torch_geometric.nn`。祝学习快乐。